<a href="https://colab.research.google.com/github/ch-kamran/ch-kamran/blob/main/NLP_02_03_23_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2023-08-02 06:06:00--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 142.251.171.128, 108.177.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2023-08-02 06:06:00 (205 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [9]:
import json
import numpy as np
import tensorflow as tf
with open('./sarcasm.json' , 'r') as f:
  data = json.load(f)

In [10]:
urls =[]
sentences =[]
labels = []
for item in data:
  urls.append(item['article_link'])
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [11]:
len(sentences)

26709

In [12]:
training_size = 20000


vocab_size = 10000
max_length = 32
embedding_dims = 16

In [24]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type = 'post'
oov_tok = '<oov>'
padding_type = 'post'

In [25]:
tokenizer= Tokenizer(num_words=vocab_size ,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
tokenizer.fit_on_texts(testing_sentences)


train_sequences=tokenizer.texts_to_sequences(training_sentences)
test_sequences=tokenizer.texts_to_sequences(testing_sentences)

train_sequences_padded=pad_sequences(train_sequences,
                                     maxlen=max_length,
                                     padding=padding_type,
                                     truncating=trunc_type)

test_sequences_padded=pad_sequences(test_sequences,
                                     maxlen=max_length,
                                     padding=padding_type,
                                     truncating=trunc_type)

training_labels = np.array(training_labels)
test_labels = np.array(testing_labels)

In [26]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dims, input_length=max_length),
    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

num_epochs = 30
hist = model.fit(train_sequences_padded, training_labels,
                 epochs=num_epochs,
                 validation_data=(test_sequences_padded, test_labels),
                 verbose=2)


Epoch 1/30
625/625 - 14s - loss: 0.5604 - accuracy: 0.7011 - val_loss: 0.3954 - val_accuracy: 0.8316 - 14s/epoch - 22ms/step
Epoch 2/30
625/625 - 4s - loss: 0.3102 - accuracy: 0.8735 - val_loss: 0.3412 - val_accuracy: 0.8520 - 4s/epoch - 6ms/step
Epoch 3/30
625/625 - 2s - loss: 0.2345 - accuracy: 0.9065 - val_loss: 0.3392 - val_accuracy: 0.8559 - 2s/epoch - 3ms/step
Epoch 4/30
625/625 - 2s - loss: 0.1899 - accuracy: 0.9267 - val_loss: 0.3542 - val_accuracy: 0.8553 - 2s/epoch - 3ms/step
Epoch 5/30
625/625 - 3s - loss: 0.1592 - accuracy: 0.9412 - val_loss: 0.3802 - val_accuracy: 0.8547 - 3s/epoch - 4ms/step
Epoch 6/30
625/625 - 3s - loss: 0.1348 - accuracy: 0.9503 - val_loss: 0.4175 - val_accuracy: 0.8489 - 3s/epoch - 4ms/step
Epoch 7/30
625/625 - 3s - loss: 0.1165 - accuracy: 0.9596 - val_loss: 0.4465 - val_accuracy: 0.8486 - 3s/epoch - 5ms/step
Epoch 8/30
625/625 - 3s - loss: 0.1006 - accuracy: 0.9661 - val_loss: 0.4881 - val_accuracy: 0.8429 - 3s/epoch - 4ms/step
Epoch 9/30
625/625 - 

In [27]:
 model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 32, 16)            160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_10 (Dense)            (None, 32)                544       
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 160,577
Trainable params: 160,577
Non-trainable params: 0
_________________________________________________________________
